# Exercise 3
## Lillian Mueller and Regina Hong 
Investigating linear regression for Iris Dataset

In [20]:
from sklearn import (
    linear_model, 
    preprocessing, 
    model_selection, 
    metrics, 
    tree)
from sklearn.datasets import load_iris
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# loading in the iris dataset from sklearn
iris_data = load_iris()

# turning dataset into dataframe format for easier reading
df_iris = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)

df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [15]:
# adding the target as a class column and renaming numbers to class names
# 0 = setosa, 1 = versicolor, 2 = virginica

df_iris['class'] = iris_data.target.tolist()
df_iris['class'].replace({0: 'setosa', 1: 'versicolor', 2: 'virginica'},
                inplace=True)

#Create a new column for the target (to be ready for processing)
label_obj = preprocessing.LabelEncoder()
df_iris['class_level'] = label_obj.fit_transform(df_iris['class'])

df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class,class_level
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
4,5.0,3.6,1.4,0.2,setosa,0


In [17]:

# Creating Train and Test datasets
predictors, target = iris_data.data, iris_data.target
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    predictors, 
    target, 
    test_size = 0.33)

# create logistic regression to classify iris
logR = linear_model.LogisticRegression().fit(x_train, y_train)

# find coefficients 
B0 = logR.intercept_[0]
B1 = logR.coef_[0][0]
print('B0: ', B0)
print('B1: ', B1)


mu = -B0/B1
s = 1/B1
print('mu: ', mu)
print('s: ', s)

B0:  9.019648180474569
B1:  -0.4203109156990072
mu:  21.459466893630985
s:  -2.379191124115652


In [32]:
# Test model with testing dataset and find accuracy 
logR_pred = logR.predict(x_test)
accuracy = {
    'LogReg' : [
        metrics.accuracy_score(y_train, logR.predict(x_train)), 
        metrics.accuracy_score(y_test, logR_pred),
        metrics.r2_score(y_test, logR_pred)
    ]
}
print(accuracy['LogReg'])

[0.99, 0.94, 0.88880652335063]


In [33]:
# create decision tree model using same training and test datasets 
dTree = tree.DecisionTreeClassifier(criterion='gini').fit(x_train, y_train)

# Test model with testing dataset and find accuracy 
dtree_pred = dTree.predict(x_test)
accuracy['DTree'] = [
        metrics.accuracy_score(y_train, logR.predict(x_train)), 
        metrics.accuracy_score(y_test, dtree_pred),
        metrics.r2_score(y_test, dtree_pred)
    ]

print(accuracy['DTree'])

[0.99, 0.92, 0.8517420311341735]


In [34]:
# Compare the accuracy of each model 
pd.DataFrame.from_dict(
    accuracy,
    orient='index', 
    columns=[
        'Train Data Accuracy', 
        'Test Data Accuracy', 
        'r2 Score'
    ])

,Train Data Accuracy,Test Data Accuracy,r2 Score
LogReg,0.99,0.94,0.888807
DTree,0.99,0.92,0.851742


In [40]:
# create ranking of results of logistic regression model 
probRanks = pd.DataFrame(
    logR.predict_proba(predictors), 
    columns=[
        'Prob of setosa (0)', 
        'Prob of versicolor (1)', 
        'Prob of virginica (2)'
    ]
)
probRanks.sample(10)

,Prob of setosa (0),Prob of versicolor (1),Prob of virginica (2)
107,0.000003,0.019870,9.801277e-01
12,0.966236,0.033764,1.645598e-07
73,0.006499,0.793583,1.999176e-01
115,0.000144,0.055155,9.447009e-01
66,0.015895,0.716980,2.671251e-01
13,0.987259,0.012741,3.313302e-08
99,0.033129,0.897922,6.894899e-02
24,0.945025,0.054973,1.303631e-06
119,0.000986,0.472746,5.262684e-01
75,0.010762,0.847755,1.414825e-01


In [44]:
# rank two new records 
records = [
    [5.8,2.8,5.1,2.4],
    [6.0,2.2,4.0,1.0]
]
newRecords = pd.DataFrame(
    records, 
    columns=[
        'sepal length (cm)',
        'sepal width (cm)',
        'petal length (cm)',
        'petal width (cm)'
    ]
)

# making prediction with model 
newRecords['Prediction'] = logR.predict(records)

# adding rankings to dataframe
rankProbs = pd.DataFrame(
    logR.predict_proba(records),
    columns=[
        'Prob of setosa (0)', 
        'Prob of versicolor (1)', 
        'Prob of virginica (2)'
        ]
    )
newRecords[rankProbs.columns] = rankProbs

newRecords

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Prediction,Prob of setosa (0),Prob of versicolor (1),Prob of virginica (2)
0,5.8,2.8,5.1,2.4,2,0.000228,0.062527,0.937245
1,6.0,2.2,4.0,1.0,1,0.016662,0.959983,0.023356
